In [1]:
import requests
import lxml.html

In [14]:
def find_entry(year,year_dict,start,end):
    '''Finds all the necessary elements to create a dictionary entry for each observation'''
    root="https://declarator.org"
    # We need to use the AJAX url to get the full list of people
    url=f"https://declarator.org/office/5/{year_dict[year]}/ajax/?start={start}&end={end}&sort_order="
    r=requests.get("http://localhost:8050/render.html",params={"url":url})
    html=lxml.html.fromstring(r.content)
    person=html.xpath('//div[@class="person_name activetext"]')
    if len(person)>0:
        person=person[0]
        person_url=root+person.xpath('./a/@href')[0]
        family_name=person.xpath('./a/div[@class="family_name"]/h4/text()')[0]
        name=person.xpath('./a/h4/text()')[0]
        position=html.xpath('//div[2]/i/text()')[0]
        party_affiliation=html.xpath('//div[@class="party"]/text()')
        if len(party_affiliation)>0:
            party_affiliation=party_affiliation[0].strip()
        else:
            party_affiliation=None
        assets=html.xpath('//p//text()')
        return {"year":year,"family_name":family_name,"name":name,"position":position,"party_affiliation":party_affiliation,"assets":assets,"url":person_url}
    else:
        return False

In [3]:
import re
def clean_text(list_of_words):
    '''Preliminary text cleaning for asset_value extraction'''
    clean_list=[re.sub(r"\n","",word).strip() for word in list_of_words]
    clean_list=[re.sub(r" m"," sqm",word).strip() for word in clean_list]
    clean_list=[item for item in clean_list if len(item)>0 and item!="2"]
    return clean_list

def clean_text_details(list_of_words):
    '''Fixes text for asset_value extraction'''
    clean_list=clean_text(list_of_words)
    new_list=[]
    i=0
    while i <=len(clean_list)-1:
        if i==len(clean_list)-1:
            new_list.append(clean_list[i])
            i+=1
        elif clean_list[i+1][0]=="(" or clean_list[i+1][0]==",":
            new_list.append(clean_list[i]+" "+clean_list[i+1])
            i+=2
        else:
            new_list.append(clean_list[i])
            i+=1
    return new_list

In [4]:
def create_dicts(year,family_name,name,title,party,url,cleaned_list):
    '''Creates clean dictionary for each entry where asset_type and asset_value are separated'''
    index_list=[]
    for item in ["Income","Real estate","Transport","Accounts"]:
        try:
            index=cleaned_list.index(item)
            index_list.append(index)
        except:
            continue
    final_output=[]
    index_list.append(len(cleaned_list))
    for i in range(len(index_list)-1):
        begin=index_list[i]+1
        end=index_list[i+1]
        span=cleaned_list[begin:end]
        for item in span:
            return_list={}
            return_list['year']=year
            return_list['last_name']=family_name
            return_list['first_name']=name
            return_list["position"]=title
            return_list["url"]=url
            return_list["party"]=party
            return_list["asset_type"]=cleaned_list[index_list[i]]
            return_list["asset_value"]=item
            final_output.append(return_list)  
    return final_output

In [5]:
url_2="https://declarator.org/office/5"
root_2="https://declarator.org"
r2=requests.get("http://localhost:8050/render.html",params={"url":url_2})
html2=lxml.html.fromstring(r2.content)
new_dict={}
for element in html2.xpath('//tr')[3:]:
    year=element.xpath('./td[position()=1]/text()')[0]
    element_2=element.xpath('./td[position()=2]//ul[@class="list-inline activetext"]/li/a')[0]
    if element_2.xpath('./text()')[0].strip()=="Anti-corruption declaration":
        anti_corrupt_url=element_2.xpath("./@href")[0]
        index_year=re.findall(r"\d{3,}",anti_corrupt_url)[0]
        new_dict[year]=index_year

In [31]:
new_dict

{'2020': '70337',
 '2019': '63283',
 '2018': '50083',
 '2017': '43675',
 '2016': '37334',
 '2015': '30984',
 '2014': '20156',
 '2013': '4786',
 '2012': '979',
 '2011': '857',
 '2009': '492'}

In [28]:
def create_entries(key,new_dict):
    all_entries=[]
    start=0
    end=1
    while find_entry(key,new_dict,start,end):
        potat=find_entry(key,new_dict,start,end)
        entries=create_dicts(potat['year'],potat['family_name'],potat['name'],potat['position'],potat['party_affiliation'],potat['url'],clean_text_details(potat['assets']))
        start+=1
        end+=1
        all_entries.extend(entries)
    return all_entries

In [36]:
# This method loops through each individual person/entry for each year. As such, it is highly inefficient as it stands. 
# Do not rerun this if you want to save time.
# The reason why this method was chosen is because in order to get the full list of people, we need to use the AJAX url. The AJAX url unfortunately has no nested HTML structure at all, making xpath very difficult.
all_entries=[]
keys=['2020','2019','2018','2017','2016','2015','2014','2013','2012','2011']
for key in keys:
    entries=create_entries(key,new_dict)
    all_entries.extend(entries)

In [55]:
import pandas as pd
output_df=pd.DataFrame(all_entries)
output_df.to_csv("Russia_council_of_federation_anti_corruption.csv",index=False)

The code below is to extract foreign name 

In [40]:
def find_russian_name(x):
    root_person="http://declarator.org/person_report/"
    id_person=re.findall(r"[0-9]+",x)[0]
    full_id_url=root_person+id_person+"/"
    df_name=pd.read_excel(full_id_url)
    return df_name['ФИО'].unique()[0]

In [41]:
output_df_3=pd.read_csv("Russia_council_of_federation_anti_corruption.csv")

In [45]:
list_of_russian_names=[]
for link in output_df_3.url.unique():
    try: 
        list_of_russian_names.append(find_russian_name(link))
    except:
        list_of_russian_names.append(None)

In [49]:
df_russian_name=pd.DataFrame({"id":output_df_3.url.unique(),"russian_name":list_of_russian_names})

In [50]:
output_df_4=output_df_3.merge(df_russian_name,how="left",right_on="id",left_on="url").drop(['id'],axis=1)

In [58]:
# Fixing mojibakes
import ftfy
output_df_4.position=output_df_4.position.apply(lambda x: ftfy.fix_encoding(x) if x is not None else None)
output_df_4.asset_value=output_df_4.asset_value.apply(lambda x: ftfy.fix_encoding(x) if x is not None else None)
output_df_4.russian_name=output_df_4.russian_name.apply(lambda x: ftfy.fix_encoding(x) if x is not None else None)

In [61]:
output_df_4.to_csv("Russia_council_of_federation_anti_corruption_full.csv",index=False)